# Memphis restaurant hunt. 


## Introduction
### 1.1 Background
Memphis is a city on the Mississippi River in southwest Tennessee, famous for the influential strains of blues, soul and rock 'n' roll that originated there. Elvis Presley, B.B. King and Johnny Cash recorded albums at the legendary Sun Studio, and Presley’s Graceland mansion is a popular attraction. The city itself has a population of 646,889. Memphis played a prominent role in the American civil rights movement and was the site of Martin Luther King Jr.'s 1968 assassination. Memphis is a regional center for commerce, education, media, art, and entertainment. There are many restaurants in Memphis, each belonging to different categories like Chinese, Italian , French etc.  As part of this project, we will list and visualize all major restaurants of Memphis.

### 1.2 Business Problem
How can entrepreneurs better understand city demographics and restaurant landscape, before opening a new restaurant?
To solve this business problem, we are going to cluster Memphis neighborhoods in order to recommend where entrepreneurs can open new venues. We will recommend arrears and highlight facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.
To explore and target recommended locations we will access data through FourSquare API interface and arrange them as a DataFrame for visualization.
Questions that can be asked using the above mentioned datasets
What area has the highest number of restaurants in Memphis? 
Which areas have fewer restaurants? 
Which areas have higher populations? 
What is the best location in Memphis for Italian restaurant?

## Data sources 
To explore and target recommended locations in Memphis restaurants we will access data through FourSquare API interface and arrange them as a DataFrame for visualization.\
For this project we need the following data:\
Data source: Fousquare API : https://developer.foursquare.com/ \
Description: Memphis restaurants data that contains list Locality, restaurant name, rating along with their latitude and longitude.\
Data source: https://www.zip-codes.com/city/tn-memphis.asp#demographics \
Description: From this source I will receive information about demographics for nearby restaurants in each locality.

In [36]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

#### Download, scrape, convert into Dataframe and clean the data
In the scraped table not all data will be used in the future work.\
I had to drop columns with irelevant information like: Country, Area code and Type.\
As scraped information was converted in Data Frame as objects we need to delete "," from the column Populations and convert objects in Integers, so we can make calculations later.

In [37]:
List_url = "https://www.zip-codes.com/city/tn-memphis.asp#demographics"
source = requests.get(List_url).text

soup = BeautifulSoup(source, 'html.parser')
memphis_data = pd.DataFrame(columns=["Zipcode", "Type", "Country", "Population", "Area Code"])
table  = soup.find('table', { "class" : "statTable"})
#print(table)
for row in table.find_all("tr"):
    col = row.find_all("td")
    #print(col)
    Zipcode =col[0].text.strip("ZIP Code")
    Type =col[1].text
    Country =col[2].text
    Population =col[3].text
    AreaCode =col[4].text
    
    
    memphis_data = memphis_data.append({"Zipcode":Zipcode, "Type":Type, "Country":Country, "Population":Population, "Area Code":AreaCode}, ignore_index=True)
memphis_data=memphis_data[memphis_data['Type']!='Type']
M_data = memphis_data.drop(columns=['Type', 'Country', 'Area Code'])
M_data = M_data.replace(',', '', regex=True)
M_data = M_data.astype(int)
M_data = M_data[M_data['Population'] !=0]

M_data.head()

,Zipcode,Population
4,38103,12180
5,38104,23409
6,38105,6184
7,38106,27222
8,38107,17698


### Now I will get the latitude and the longitude coordinates of each neighborhoods and combine different neighborhoods that exist in one postal code area.

In [38]:
ZIP = pd.read_csv('https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/download/?format=csv&timezone=Europe/Helsinki&lang=en&use_labels_for_header=true&csv_separator=%3B', sep=';')
ZIP.head()

,Zip,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
0,55795,Willow River,MN,46.317812,-92.84315,-6,1,"46.317812,-92.84315"
1,45388,Yorkshire,OH,40.328535,-84.47938,-5,1,"40.328535,-84.47938"
2,39483,Foxworth,MS,31.218509,-89.90761,-6,1,"31.218509,-89.90761"
3,31503,Waycross,GA,31.205194,-82.37534,-5,1,"31.205194,-82.37534"
4,45833,Delphos,OH,40.841409,-84.34178,-5,1,"40.841409,-84.34178"


### The dataframe consist of columns: ZipCode, City, State, geopoint, Timezone, Daylight savings time flag.
### For the mean of this assighnment I will drop columns with the information I do not need: City, State, geopoint, Timezone, Daylight savings time flag.
### And I will merge to geo data with ZIP Codes.

In [39]:
ZIP1 = ZIP.drop(columns=['geopoint', 'Timezone', 'Daylight savings time flag', 'State', 'City'])
ZIP1.rename(columns={'Zip':'Zipcode'},inplace=True)
geo_merged = pd.merge(ZIP1, M_data, on='Zipcode')
geo_data=geo_merged[['Zipcode','Population','Latitude','Longitude']]
geo_data.head()


    Zipcode  Population   Latitude  Longitude
0     38119       22330  35.082936 -89.848920
1     38117       26125  35.112929 -89.903890
2     38134       41948  35.193046 -89.865590
3     38127       45248  35.223796 -90.006460
4     38152         834  35.201738 -89.971538
5     38108       18729  35.174165 -89.960280
6     38112       17472  35.148605 -89.974790
7     38109       46594  35.050286 -90.075230
8     38118       41465  35.048455 -89.924260
9     38138       24045  35.088885 -89.806770
10    38126        7334  35.126469 -90.043590
11    38107       17698  35.167515 -90.022240
12    38103       12180  35.146131 -90.053400
13    38114       26905  35.101525 -89.985430
14    38133       20787  35.208709 -89.805180
15    38132           7  35.073545 -89.997880
16    38116       40404  35.033190 -90.011280
17    38111       41742  35.109350 -89.943630
18    38141       22462  35.016803 -89.847010
19    38139       16086  35.087468 -89.761500
20    38122       25270  35.155589

I choose only boroughs that contain the word Toronto.



## Problem 3
### Explore and cluster the neighborhoods in Toronto.


In [5]:
import numpy as np
import os
from sklearn.cluster import KMeans
!pip install folium
import folium 
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 94 kB 6.0 MB/s  eta 0:00:01


In [6]:
CLIENT_ID = 'G1IRT315WKKXPWVBY50H0KTZ0IVLULFI1HGRKRK4OXN55CWB' # your Foursquare ID
CLIENT_SECRET = 'I3QMFSSLQF5OGVHSVASJ5VXMDIK1IPVETVIMBNUZMR0RJVOT' # your Foursquare Secret
VERSION = '20180604'

### Now I will create a list and dataframe with the venues in the area with raius 500.

In [40]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=1000
    LIMIT=200
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Zipcode', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [41]:
Memphis_venues = getNearbyVenues(names=geo_data['Zipcode'],
                                   latitudes=geo_data['Latitude'],
                                   longitudes=geo_data['Longitude']
                                  )


38119
38117
38134
38127
38152
38108
38112
38109
38118
38138
38126
38107
38103
38114
38133
38132
38116
38111
38141
38139
38122
38128
38125
38115
38135
38104
38105
38120
38106


In [43]:
Memphis_venues['Venue Category'].unique()

array(['Intersection', 'Kitchen Supply Store',
       'Southern / Soul Food Restaurant', 'Big Box Store',
       'Sushi Restaurant', 'American Restaurant', 'Restaurant', 'Theater',
       'Bookstore', 'Burger Joint', 'Gym / Fitness Center',
       'Grocery Store', 'Bagel Shop', 'Fast Food Restaurant', 'Pet Store',
       'Coffee Shop', 'Health & Beauty Service', 'Italian Restaurant',
       'BBQ Joint', 'Frozen Yogurt Shop', 'Breakfast Spot', 'Café',
       'Fried Chicken Joint', 'Bakery', 'Donut Shop', 'Bar', 'Boutique',
       'Seafood Restaurant', 'Wine Shop', 'Wings Joint', 'Ice Cream Shop',
       'Chinese Restaurant', 'Furniture / Home Store', 'Supermarket',
       'Middle Eastern Restaurant', 'Clothing Store', 'Department Store',
       'Deli / Bodega', 'Mexican Restaurant', 'Nail Salon', 'Candy Store',
       'Sporting Goods Shop', 'Park', 'Pizza Place', 'Shipping Store',
       'Sandwich Place', 'Liquor Store', 'Cosmetics Shop',
       'Food & Drink Shop', 'Pharmacy', 'Movie T

In [46]:
Italian_Restaurant = Memphis_venues[Memphis_venues['Venue Category']=='Italian Restaurant']
print(Italian_Restaurant)

     Zipcode  Neighborhood Latitude  Neighborhood Longitude  \
18     38117              35.112929               -89.90389   
312    38103              35.146131               -90.05340   
458    38111              35.109350               -89.94363   

                        Venue  Venue Latitude  Venue Longitude  \
18   Ciao Bella Italian Grill       35.115339       -89.898815   
312      Catherine and Mary's       35.138775       -90.055667   
458            Pete and Sam's       35.107960       -89.933031   

         Venue Category  
18   Italian Restaurant  
312  Italian Restaurant  
458  Italian Restaurant  


### Now I can count them.

In [26]:
Memphis_venues.groupby('Zipcode').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Zipcode,,,,,,
38103,40,40,40,40,40,40
38104,30,30,30,30,30,30
38105,2,2,2,2,2,2
38106,2,2,2,2,2,2
38108,2,2,2,2,2,2
38111,14,14,14,14,14,14
38112,24,24,24,24,24,24
38114,11,11,11,11,11,11
38115,4,4,4,4,4,4


### and now I group them and identify the most common venue

In [19]:
Memphis_onehot = pd.get_dummies(Memphis_venues[['Venue Category']], prefix="", prefix_sep="")

Memphis_onehot.insert(loc=0, column='Zipcode', value=Memphis_venues['Zipcode'] )
Memphis_grouped = Memphis_onehot.groupby('Zipcode').mean().reset_index()
Memphis_grouped.head()

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Zipcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Zipcode'] = Memphis_grouped['Zipcode']

for ind in np.arange(Memphis_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Memphis_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Zipcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,38103,Hotel,Harbor / Marina,Salon / Barbershop,Bar,Park,Burger Joint,Grocery Store,Pub,Pizza Place,Nightclub
1,38104,Mobile Phone Shop,Coffee Shop,Wings Joint,Ice Cream Shop,Sandwich Place,Pizza Place,Pharmacy,Middle Eastern Restaurant,Lounge,Japanese Restaurant
2,38105,Gym / Fitness Center,BBQ Joint,Wings Joint,Food,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store
3,38106,Convenience Store,Park,Home Service,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Disc Golf
4,38108,Light Rail Station,Wings Joint,Food,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Disc Golf


### Now I will disstribute venues in different Clusters for Neighborhoods

In [20]:
# set number of clusters
kclusters = 5

Memphis_grouped_clustering = Memphis_grouped.drop('Zipcode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Memphis_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 4, 1, 0, 0, 0, 4, 2], dtype=int32)

### I will add clusters in dataframe

In [21]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Memphis_merged = geo_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Memphis_merged = Memphis_merged.join(neighborhoods_venues_sorted.set_index('Zipcode'), on='Zipcode')

Memphis_merged.head()

,Zipcode,Population,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,38116,40404,35.033190,-90.01128,2.0,Bus Stop,Wings Joint,Construction & Landscaping,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Disc Golf
1,38111,41742,35.109350,-89.94363,0.0,Pharmacy,Fried Chicken Joint,Ethiopian Restaurant,Video Store,Coffee Shop,Grocery Store,Asian Restaurant,Burger Joint,Gas Station,Seafood Restaurant
2,38141,22462,35.016803,-89.84701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,38126,7334,35.126469,-90.04359,0.0,Southern / Soul Food Restaurant,Gas Station,Dance Studio,Buffet,Music Venue,Fast Food Restaurant,Convenience Store,Gastropub,Gift Shop,Cosmetics Shop
4,38119,22330,35.082936,-89.84892,0.0,Home Service,Photography Studio,Shop & Service,Mobile Phone Shop,Fast Food Restaurant,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega


#### and sort them

In [22]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Zipcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,38103,Hotel,Harbor / Marina,Salon / Barbershop,Bar,Park,Burger Joint,Grocery Store,Pub,Pizza Place,Nightclub
1,0,38104,Mobile Phone Shop,Coffee Shop,Wings Joint,Ice Cream Shop,Sandwich Place,Pizza Place,Pharmacy,Middle Eastern Restaurant,Lounge,Japanese Restaurant
2,0,38105,Gym / Fitness Center,BBQ Joint,Wings Joint,Food,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store
3,4,38106,Convenience Store,Park,Home Service,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Disc Golf
4,1,38108,Light Rail Station,Wings Joint,Food,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Disc Golf


In [23]:
address = 'Memphis, TN'

geolocator = Nominatim(user_agent="Mem_finder")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Memphis downtown are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Memphis downtown are 35.1490215, -90.0516285.


### I will put all the venuse on the map with different colors.

In [29]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Memphis_merged['Latitude'], Memphis_merged['Longitude'], Memphis_merged['Zipcode'], Memphis_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        color=rainbow[cluster-1],
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

TypeError: list indices must be integers or slices, not float